# 误差补偿
本笔记本介绍了从数据分析、数据清洗、模型训练、模型部署和测试的一套完整流程，并且提供了一套基本的误差补偿模型。根据指令对本笔记本中的相应代码区域进行修改并执行，即可生成并且部署一套误差补偿服务。参赛选手需要根据实际的数据情况，对模型进行调优。


## 数据加载
我们的误差补偿训练数据存放在<I>adjustments.tsv</I>中，测试数据放在<I>test_adjustments.tsv</I>。

**测试数据集为真实值，不能进行调整，否则会导致实际模型测试结果和实际的预测结果存在偏差，出现过于乐观或者过于消极的测试结果**

只需要执行加载训练数据的代码即可，当然也可以修改代码加载指定文件

In [1]:
import numpy as np
import pandas as pd
# 加载训练数据
train_dataset = pd.read_csv('./adjustments.tsv',
                          sep='\t',
                          skipinitialspace=True)
# 加载测试数据     ！！！ 测试数据集为真实值，不能进行调整，否则将会导致实际模型测试结果和真实预测结果存在偏差，使得最终加工的作品和预期不一致
test_dataset = test_dataset = pd.read_csv('./test_adjustments.tsv',
                          sep='\t',
                          skipinitialspace=True)

## 数据分析

打印并查看数据，可以看出，adjustments.tsv文件的前n列为特征值，这些值代表着真实世界中影响机床的环境因素，例如刀具磨损、温度、湿度等等；后面几列为补偿指令。

In [2]:
np.set_printoptions(precision=3, suppress=True)
dataset = train_dataset.copy()
dataset.head()

,特征0,特征1,特征2,特征3,特征4,特征5,特征6,特征7,特征8,特征9,...,特征16,特征17,补偿0,补偿1,补偿2,补偿3,补偿4,补偿5,补偿6,补偿7
0,1.4158,2.9711,10.7935,7.5279,2.3352,8.1042,2.3096,3.3367,11.8639,12.7142,...,171.764,1434.24,0.331511,-0.932553,0.285048,-0.1435,-0.833982,0.767568,0.463969,1.904800
1,0.6280,1.8616,10.1770,7.4684,2.1915,8.5945,0.1379,2.9661,11.5816,12.2487,...,185.824,1469.19,0.894066,-0.446796,0.058519,-0.4624,0.715252,0.999105,0.988844,0.689742
2,0.9648,1.8103,10.1682,5.9705,2.0629,6.5349,2.8694,3.1185,11.7464,12.2074,...,187.576,1540.76,0.999982,0.460716,0.997809,-0.4624,0.723031,0.992935,0.682903,0.749580
3,0.7119,1.6221,10.1487,6.8678,2.0694,6.8806,1.5791,2.3003,11.5545,12.0659,...,189.938,1498.29,0.998794,-0.862448,0.329694,-0.4624,0.891745,0.015078,0.997127,0.984439
4,0.3797,1.6852,10.9601,5.0035,3.1659,5.9471,0.0858,2.6402,11.7458,12.7041,...,181.275,1465.11,0.442893,-0.990703,-0.151400,0.2245,0.617880,-0.506397,-0.997502,0.376126


然后我们可以简单分析数据，比如我们可以查看特征值和补偿值的分布特性，比如均值和方差：

In [3]:
average = np.average(dataset.values[:,:18], axis=0)
variance = np.var(dataset.values[:,:18], axis=0)
print('均值', average)
print('方差', variance)

均值 [   1.       1.998   10.5      6.998    3.001    6.995    2.005    2.999
   11.736   12.604    1.768    0.649   19.417   19.598    3.349   17.983
  172.057 1475.054]
方差 [   0.332    0.332    0.083    1.327    0.333    1.337    1.331    0.336
    0.017    0.039    0.074    0.213    0.163    0.069   10.31   525.861
  149.833 3136.668]


## 构建训练集和测试集

接着，我们将数据分为训练集和测试集

我们分别获取训练集和测试集的特征以及补偿值：

In [5]:
train_features = dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features[['补偿'+str(i) for i in range(8)]].copy()
test_labels = test_features[['补偿'+str(i) for i in range(8)]].copy()

train_features = train_features.drop(['补偿'+str(i) for i in range(8)], axis=1)
test_features = test_features.drop(['补偿'+str(i) for i in range(8)], axis=1)

print(train_features.shape, train_labels.shape)
print(test_features.shape, test_labels.shape)

(50000, 18) (50000, 8)
(2998, 18) (2998, 8)


数据的分析和预处理的方法有很多种，我们只展示了一种方法。用户可根据自己的需要使用其他方法。

数据的分析和预处理的方法有很多种，我们只展示了一种方法。用户可根据自己的需要使用其他方法。

## 模型构建

本平台支持基于Tensorflow-Serving的HTTP调用方式：该方式支持任何部署在TensorFlow Serving上的模型

### TensorFlow
首先，我们导入相关的依赖包。

In [6]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

print(tf.__version__)

2.8.2


然后，我们开始构建模型

In [8]:
model = tf.keras.Sequential([ 
    layers.Dense(100, input_dim=train_features.shape[1], activation="relu"),
    layers.Dense(train_labels.shape[1])
])
model.compile(loss="mse", optimizer="adam") # 根据情况调整参数
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               1900      
                                                                 
 dense_3 (Dense)             (None, 8)                 808       
                                                                 
Total params: 2,708
Trainable params: 2,708
Non-trainable params: 0
_________________________________________________________________


## 模型训练
设置模型训练参数进行模型训练

In [11]:
from sklearn.metrics import mean_squared_error

model.fit(   # 根据情况调整参数
    train_features,
    train_labels,
    epochs=50,
    batch_size=32
)

Epoch 1/50
1563/1563 [==============================] - 2s 989us/step - loss: 0.3926
Epoch 2/50
1563/1563 [==============================] - 2s 984us/step - loss: 0.3906
Epoch 3/50
1563/1563 [==============================] - 2s 1ms/step - loss: 0.3949
Epoch 4/50
1563/1563 [==============================] - 2s 996us/step - loss: 0.3832
Epoch 5/50
1563/1563 [==============================] - 2s 1ms/step - loss: 0.3835
Epoch 6/50
1563/1563 [==============================] - 2s 1ms/step - loss: 0.3798
Epoch 7/50
1563/1563 [==============================] - 2s 1ms/step - loss: 0.3688
Epoch 8/50
1563/1563 [==============================] - 2s 1ms/step - loss: 0.3704
Epoch 9/50
1563/1563 [==============================] - 2s 1ms/step - loss: 0.3666
Epoch 10/50
1563/1563 [==============================] - 2s 1ms/step - loss: 0.3484
Epoch 11/50
1563/1563 [==============================] - 2s 1ms/step - loss: 0.3518
Epoch 12/50
1563/1563 [==============================] - 2s 1ms/step - loss: 0.

测试模型训练结果

In [12]:
test_preds = model.predict(test_features)
print("y1 MSE:%.4f" % mean_squared_error(test_labels, test_preds))
print("y1 MSE:%.4f" % len(test_labels), '------',len(test_preds))

y1 MSE:0.2626
y1 MSE:2998.0000 ------ 2998


### 模型部署

误差补偿模型的部署路径为<I>v1/models/slot0/versions/<版本号>/</I> ，且版本号必须为数字。注意，tensorflow-serving在加载模型的时候会自动加载版本号最高的模型，并卸载低版本号的模型。因此，每次部署新部署模型时需要递增版本号。由于我们的系统已经预置了一个低精度版本的模型，并且将版本号设置为1，所以用户在部署自定义模型时应当至少将版本号设置为2。

In [20]:
model_version = 3
tf.keras.models.save_model(
    model,
    f'/models/slot0/{model_version}/', # v1/models/slot0/为tensorflow-serving的模型根目录
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

INFO:tensorflow:Assets written to: /models/slot0/3/assets


注意，tensorflow-serving卸载旧版本模型并加载新版本模型的过程往往需要数十秒的时间，在次期间对模型发送请求会得到“Servable not found for request”的错误。用户可以使用<I>docker logs adjustment-serving-container</I>查看是否已经加载完毕。

接下来我们测试是否部署成功：

In [18]:
import json
import requests
from pprint import pprint

req_data = json.dumps({
            'inputs': test_features.values[:1].tolist()
        })  
print(req_data)
response = requests.post(f'http://fireeye-test-model-container:8501/v1/models/slot0/versions/{model_version}:predict', # 根据部署地址填写
                         data=req_data,
                         headers={"content-type": "application/json"})
if response.status_code != 200:
    raise RuntimeError('Request tf-serving failed: ' + response.text)
resp_data = json.loads(response.text)    
if 'outputs' not in resp_data \
                    or type(resp_data['outputs']) is not list:
    raise ValueError('Malformed tf-serving response')

print(resp_data)
print("{'outputs':",test_labels.values[:1].tolist())

print("y1 MSE:%.4f" % mean_squared_error(test_labels.values[:1].tolist(), resp_data['outputs']))


{"inputs": [[1.4158, 2.9711, 10.7935, 7.5279, 2.3352, 8.1042, 2.3096, 3.3367, 11.8639, 12.7142, 1.8581, 0.3898, 19.8309, 19.771, 0.0001, 1.7768, 171.764, 1434.24]]}
{'outputs': [[0.289782584, -0.678707659, 0.587964356, 0.0605452359, -1.20483196, 0.647470057, 0.340225428, 1.0843271]]}
{'outputs': [[0.331511, -0.932553, 0.285048, -0.1435, -0.833982, 0.767568, 0.463969, 1.9048]]
y1 MSE:0.1300


测试成功之后，用户需要在web页面配置相关任务的服务地址，地址的格式为：<I>“http://fireeye-test-model-container:8501/v1/models/slot0/versions/<版本号>:predict ”</I>。